In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'quanta.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

### Image as marker with bokeh

```Py

p = figure(x_range=(0,500), y_range=(0,500))
p.image_url(url=[ "http://pngimg.com/uploads/palm_tree/palm_tree_PNG2504.png", 
                  "https://www.referenseo.com/wp-content/uploads/2019/03/image-attractive.jpg"],
             x=[1, 5], y=[1, 5], w=253, h=409, anchor="bottom_left", alpha=0.3)

In [3]:
from apps.inventories.models import Stock

In [4]:
df = Stock.objects.to_dataframe(index="id", fieldnames=["product_item__product__name", "location", "units", "units_sold", "product_item__attribute_values__attribute", "product_item__attribute_values__value", "expiration_date"])

In [5]:
df

,product_item__product__name,location,units,units_sold,product_item__attribute_values__attribute,product_item__attribute_values__value,expiration_date
id,,,,,,,
1,Chemise camerounaise,Store 1,0,30,Fashion | color,red,2023-10-10 00:00:00+00:00
2,Chemise camerounaise,Store 2,50,56,Fashion | color,blue,2023-10-10 00:00:00+00:00
3,Chemise camerounaise,Store 1,1000,30,Fashion | color,red,2022-10-10 00:00:00+00:00
4,Lean django by liedji,Dschang,9,6,document | editor,red,2023-10-10 00:00:00+00:00


In [6]:
from bokeh.models import ColumnDataSource
cds = ColumnDataSource(data=df)

In [7]:
from bokeh.plotting import figure, show
p = figure(title="Stock data") 
p.line(x="id", y="units_sold", source=cds)

GlyphRenderer(id='p1058', ...)

In [8]:
show(p)

In [9]:
from bokeh.embed import components
script, div = components(p)

In [10]:
stock_history = Stock.history.all().order_by('-history_date')

In [11]:
for history_item in stock_history:
    print(history_item.history_date.strftime('%Y-%m-%d'), 
          history_item.history_type, 
          history_item.units, 
          history_item.history_change_reason, 
          history_item.product_item.product.name, 
          history_item.location, 
          history_item.organization, 
          history_item.history_user.username)
    

2022-12-27 ~ 1000 None Chemise camerounaise Store 1 creditium admin
2022-12-27 + 9 None Lean django by liedji Dschang Hetelo admin
2022-12-26 + 0 None Chemise camerounaise Store 1 creditium admin
2022-12-26 + 50 None Chemise camerounaise Store 2 creditium admin
2022-12-26 + 0 Add a question Chemise camerounaise Store 1 creditium admin


In [12]:
stock_history.filter(
    history_user__username="admin", 
                     history_type="+" ).order_by("-history_date").count()

4

In [13]:
stock = Stock.objects.first()

In [14]:
from simple_history.utils import update_change_reason

In [15]:
update_change_reason(stock, 'Add a question')

In [16]:
stock.history.last().history_change_reason

'Add a question'

In [17]:
from apps.organization.models import Organization
stock.history.filter(organization=Organization.objects.first())

<HistoricalQuerySet []>

In [18]:
Stock.history_for_organization(organization="creditium")

<HistoricalQuerySet [<HistoricalStock: Stock object (3) as of 2022-12-27 17:01:29.968771+00:00>, <HistoricalStock: Stock object (3) as of 2022-12-26 19:16:49.562187+00:00>, <HistoricalStock: Stock object (2) as of 2022-12-26 19:13:42.819570+00:00>, <HistoricalStock: Stock object (1) as of 2022-12-26 19:11:43.329921+00:00>]>

In [19]:
stocks = Stock.objects.for_organization("creditium")

In [20]:
from django.db.models.aggregates import Sum

In [21]:
stocks_units_by_location = stocks.aaggregate(Sum('units'))

In [22]:
stocks_units_by_location

<coroutine object QuerySet.aaggregate at 0x7fb1245f0940>

In [23]:
from apps.inventories.models import Location

In [24]:
locations = Location.objects.for_organization(organization="creditium")

In [25]:
Stock.objects.for_organization(organization="creditium").filter(location__in=locations).count()

3

In [26]:
for location in locations:
    print(location.stocks.aggregate(units=Sum("units")))

{'units': 1000}
{'units': 50}


In [27]:
locations = Location.objects.for_organization(organization="creditium").annotate(units=Sum("stocks__units")).order_by("units")

In [28]:
for location in locations:
    print(location.units)

50
1000


In [29]:
df = locations.to_dataframe(index="name", fieldnames=["name","units"])
cds = ColumnDataSource(df)

In [30]:
df

,units
name,
Store 2,50
Store 1,1000


In [36]:
from bokeh.plotting import figure, show

In [37]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Bright6
from bokeh.transform import factor_cmap
p = figure(x_range=cds.data["name"], y_range=(0,1400), 
           height=350, title="Stock inventory per location")

p.vbar(x='name', top='units', width=0.9, legend_field="name",
       line_color='white', 
       fill_color=factor_cmap('name', palette=Bright6, factors=cds.data["name"]),
       source=cds)
p.toolbar.logo=None
p.title.align = "center"
p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"
show(p)




RuntimeError: Models must be owned by only a single document, FactorRange(id='p1209', ...) is already in a doc